# Drop into Berlin - Google Places API Scraping Notebook

In [ ]:
#Installation of neccessary libraries
!pip install googlemaps

In [ ]:
#Installation of neccessary libraries
!pip install pgeocode

In [ ]:
!pip install gomaps

In [8]:
#Import neccessary libraries
import os
import requests
import googlemaps
import json
from gomaps import maps_search
import time
import googlemaps
import pandas as pd
import pgeocode
import requests
import json

## Scraping Methodology 1: Specify soly solo coordinates of Berlin city

In [9]:
#Function to transform miles to meter
def miles_to_meters(miles):
    try:
        return miles * 1_609.344
    except:
        return 0

In [10]:
#Setup of API Client
api_key = open("personal_path/key.txt").read()
map_client = googlemaps.Client(api_key)

#Define coordinates of Berlin as location,from which restaurant data should be extracted from API
(lat, lng) = "52.520008","13.404954"

#Specify type of data, that should be extracted from API. In the first place, the Drop into Berlin should be focused on restaurant recommendations only
search_string = 'restaurant'

#Define the distance (i.e. radius) from the coordinates, around which data should be extracted from
distance = miles_to_meters(1)

#Setup of API request and response
response = map_client.places_nearby(location=(lat, lng), keyword=search_string, radius=distance)

In [255]:
#Looping over API respone and inserting results into dataframe
#Next_page_toke = "Click on the next page button"
business_list = []
business_list.extend(response.get('results'))
next_page_token = response.get('next_page_token')

#While there is still the option next_page_token present, another request is being sent to the Google Places API and then appended to the "business_list"-list
while next_page_token:
    time.sleep(2)
    response = map_client.places_nearby(location=(lat, lng), keyword=search_string, radius=distance, page_token=next_page_token)   
    business_list.extend(response.get('results'))
    next_page_token = response.get('next_page_token')

#Transform the list into a pandas dataframe
df = pd.DataFrame(business_list)

#Insert new column "url" (of the restaurant) to the dataframe
df['url'] = 'https://www.google.com/maps/place/?q=place_id:' + df['place_id']

In [ ]:
#Sample response
response["results"][3]

In [ ]:
#Store extracted data into xls file
df.to_excel('{0}_batch1.xlsx'.format(search_string), index=False)

#Problem: The main limitation of the previous methodology of scraping restaurant data in Berlin was that only 500-600 restaurants were being extracted out of approximately 15,000 restaurants using the main coordinates of Berlin ((lat, lng) = "52.520008","13.404954"). 
#Solution: To overcome this limitation, a solution was implemented in which separate API requests were sent for each zip code in Berlin. The aim of this approach was to extract restaurant data that is spread across the entire city of Berlin.

## Scraping Methodology 2: Specify multiple coordinates - based on zip codes - of Berlin city

In [ ]:
#Sourcing zip codes of each district of berlin to evenly scrape data across berlin areas
zip_codes = pd.read_excel('berlin_zip_codes.xlsx', sheet_name='Tabelle1')

In [70]:
#Extract longitude out of zip_codes file
zip_codes["longitude"] = [nomi.query_postal_code(x)["longitude"] for x in zip_codes["zip_code"]]

In [69]:
#Extract latitude out of zip_codes variable
zip_codes["latitude"] = [nomi.query_postal_code(x)["latitude"] for x in zip_codes["zip_code"]]

In [101]:
#Preview final zip codes of Berlin
zip_codes.head()

,zip_code,city,longitude,latitude
0,10115,Berlin,13.3846,52.5323
1,10117,Berlin,13.3872,52.5170
2,10119,Berlin,13.4053,52.5305
3,10178,Berlin,13.4096,52.5213
4,10179,Berlin,13.4164,52.5122
...,...,...,...,...
185,14169,Berlin,13.2573,52.4496
186,14193,Berlin,13.2365,52.4831
187,14195,Berlin,13.2829,52.4589
188,14197,Berlin,13.3118,52.4734


In [94]:
#Store longitude and latitude data of each zip code in list
locations = []
for i in range(len(zip_codes)):
    locations.append([zip_codes.iloc[i,2], (zip_codes.iloc[i,3])])

In [190]:
#Iterate again over different zip codes of berlin and for each zip code conduct API data request
business_list = []
for location in locations:
    lng = location[0]
    lat = location[1]
    response = map_client.places_nearby(location=(lat, lng), keyword=search_string, radius=distance)
    business_list.extend(response.get('results'))
    next_page_token = response.get('next_page_token')
    while next_page_token:
        time.sleep(2)
        response = map_client.places_nearby(location=(lat, lng), keyword=search_string, radius=distance, page_token=next_page_token)   
        business_list.extend(response.get('results'))
        next_page_token = response.get('next_page_token')
        
#Transform the list into a pandas dataframe
df = pd.DataFrame(business_list)

#Insert new column "url" (of the restaurant) to the dataframe
df['url'] = 'https://www.google.com/maps/place/?q=place_id:' + df['place_id']

In [192]:
#Store extracted data into xls file
df.to_excel('{0}_batch2.xlsx'.format(search_string), index=False)

In [198]:
#Readin first batch xls file
batch1 = pd.read_excel("Data/restaurant_batch1.xlsx")

In [199]:
#Readin second batch xls file
batch2 = pd.read_excel("Data/restaurant_batch2.xlsx")

In [200]:
#Concatenate batches
total = pd.concat([batch1, batch2], axis=0, ignore_index = True)

In [210]:
#Drop duplicates of the concatenated dataframe
total.drop_duplicates(subset=["place_id"], inplace = True)

In [ ]:
#Note: There exit two level of details per API request
# First level being extracted via coordinates entails high level information per location (=total)
# Second level being extracted via place_ids entails detailed information per location (=total_details)

#Final high level restaurant dataframe w/o further deep dive details (e.g. reviews etc.) (=Level 1)
total.head()

In [286]:
#Level 2 Data Scraping
#Extract all place_ids and put it in iterable list
place_ids = list(total["place_id"].unique())

In [ ]:
#Code to extract for all place_ids more detailed information of the google maps api

#Definition of API key 
api_key = open("personal_path/key.txt").read()

#Definition of list of place IDs for the places I want to retrieve details for
place_ids

# Initialize an empty list to store the place details
place_details_list = []

# Loop through the list of place IDs and retrieve the details for each place
for place_id in place_ids:
    # Define the API endpoint for retrieving place details
    url = f'https://maps.googleapis.com/maps/api/place/details/json?place_id={place_id}&key={api_key}'

    # Send a GET request to the API endpoint and retrieve the response
    response = requests.get(url)

    # Parse the JSON response and convert it to a dictionary
    result = json.loads(response.text)

    try:
        place_details_list.append(result["result"])
    
    except:
        pass

In [311]:
#Convert the details into a dataframe
total_details = pd.DataFrame(data = place_details_list)

In [291]:
#Save the created dataframe into a file
total_details.to_excel('{0}_place_id_details.xlsx'.format(search_string), index=False)

In [313]:
#Merge the high level (=level 1) as well as detailed information (=level 2) for each place_id into one single dataframe
restaurants_final = pd.concat([total.set_index("place_id"), total_details.set_index("place_id")], axis = 1, join="inner")

In [315]:
#Save the concatenated dataframe into a xlsx file
restaurants_final.to_excel('database', index=False)